In [ ]:
import keyring
import getpass
import functools

import pydicom

import segments

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._experimental.autosegmentation import pipeline

In [ ]:
if not keyring.get_password('segments-ai', 'api-key'):
    segments_api_key = getpass.getpass()
    keyring.set_password('segments-ai', 'api-key', segments_api_key)

In [ ]:
if not keyring.get_password('github', 'api-key'):
    github_api_key = getpass.getpass()
    keyring.set_password('github', 'api-key', github_api_key)

In [ ]:
ct_uid = '1.2.840.113704.1.111.3096.1537312918.112198'

In [ ]:
structures_to_learn = [
    'lens_left', 'lens_right', 'eye_left', 'eye_right', 'patient']

In [ ]:
(
    data_path_root,
    structure_set_paths,
    ct_image_paths,
    ct_uid_to_structure_uid,
    structure_uid_to_ct_uids,
    names_map,
    structure_names_by_ct_uid,
    structure_names_by_structure_set_uid,
    uid_to_url,
    hash_path,
) = pipeline.get_dataset_metadata()

In [ ]:
@functools.lru_cache()
def get_dcm_ct_from_uid(ct_uid):
    ct_path = ct_image_paths[ct_uid]
    dcm_ct = pydicom.read_file(ct_path, force=True)

    dcm_ct.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

    return dcm_ct

@functools.lru_cache()
def get_dcm_structure_from_uid(structure_set_uid):
    structure_set_path = structure_set_paths[structure_set_uid]

    dcm_structure = pydicom.read_file(
        structure_set_path,
        force=True,
        specific_tags=["ROIContourSequence", "StructureSetROISequence"],
    )

    return dcm_structure

@functools.lru_cache()
def get_contours_by_ct_uid_from_structure_uid(structure_set_uid):
    dcm_structure = get_dcm_structure_from_uid(structure_set_uid)

    number_to_name_map = {
        roi_sequence_item.ROINumber: names_map[roi_sequence_item.ROIName]
        for roi_sequence_item in dcm_structure.StructureSetROISequence
        if names_map[roi_sequence_item.ROIName] is not None
    }

    contours_by_ct_uid = pipeline.get_contours_by_ct_uid(dcm_structure, number_to_name_map)

    return contours_by_ct_uid

In [ ]:
structure_uid = ct_uid_to_structure_uid[ct_uid]

In [ ]:
ct_path = pipeline.download_uid(data_path_root, ct_uid, uid_to_url, hash_path)
structure_path = pipeline.download_uid(data_path_root, structure_uid, uid_to_url, hash_path)

In [ ]:
dcm_ct = get_dcm_ct_from_uid(ct_uid)
dcm_structure = get_dcm_structure_from_uid(structure_uid)

In [ ]:
grid_x, grid_y, ct_img = pipeline.create_input_ct_image(dcm_ct)

In [ ]:
contours_by_ct_uid = get_contours_by_ct_uid_from_structure_uid(
    structure_uid
)

In [ ]:
# contours_by_ct_uid

In [ ]:
contours_on_this_slice = contours_by_ct_uid[ct_uid].keys()
contours_on_this_slice

In [ ]:
GH_API = "https://api.github.com"
GH_REPO = f"{GH_API}/repos/pymedphys/data"
GH_TAGS = f"{GH_API}/releases/tags/segments.ai"
AUTH = f"Authorization: token {github_api_key}"

In [ ]:
# repo = sys.argv[1]
# tag = sys.argv[2]
# upload_file = sys.argv[3]

# token = os.environ['GITHUB_TOKEN']

# url_template = 'https://{}.github.com/repos/' + repo + '/releases'

# # Create.
# _json = json.loads(urlopen(Request(
#     url_template.format('api'),
#     json.dumps({
#         'tag_name': tag,
#         'name': tag,
#         'prerelease': True,
#     }).encode(),
#     headers={
#         'Accept': 'application/vnd.github.v3+json',
#         'Authorization': 'token ' + token,
#     },
# )).read().decode())
# release_id = _json['id']

# # Upload.
# with open(upload_file, 'br') as myfile:
#     content = myfile.read()
# _json = json.loads(urlopen(Request(
#     url_template.format('uploads') + '/' + str(release_id) + '/assets?' \
#       + urlencode({'name': os.path.split(upload_file)[1]}),
#     content,
#     headers={
#         'Accept': 'application/vnd.github.v3+json',
#         'Authorization': 'token ' + token,
#         'Content-Type': 'application/zip',
#     },
# )).read().decode())